In [49]:
import sys
sys.path.append("../src")
import utils
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet

Step One: Load the raw data, no nan filling or one hot encoding.

In [50]:
train = utils.get_train_data()

How to handle nans

In [51]:
def do_nan(my_frame, option):
    if(option == "zeroes"):
        return my_frame.fillna(0)
    if(option == "custom"):
        # MSSubClass as str
        features = my_frame
        features['MSSubClass'] = features['MSSubClass'].astype(str)

        # MSZoning NA in pred. filling with most popular values
        features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])

        # LotFrontage  NA in all. I suppose NA means 0
        features['LotFrontage'] = features['LotFrontage'].fillna(features['LotFrontage'].mean())

        # Alley  NA in all. NA means no access
        features['Alley'] = features['Alley'].fillna('NOACCESS')

        # Converting OverallCond to str
        features.OverallCond = features.OverallCond.astype(str)

        # MasVnrType NA in all. filling with most popular values
        features['MasVnrType'] = features['MasVnrType'].fillna(features['MasVnrType'].mode()[0])

        # BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
        # NA in all. NA means No basement
        for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
            features[col] = features[col].fillna('NoBSMT')

        # TotalBsmtSF  NA in pred. I suppose NA means 0
        features['TotalBsmtSF'] = features['TotalBsmtSF'].fillna(0)

        # Electrical NA in pred. filling with most popular values
        features['Electrical'] = features['Electrical'].fillna(features['Electrical'].mode()[0])

        # KitchenAbvGr to categorical
        features['KitchenAbvGr'] = features['KitchenAbvGr'].astype(str)

        # KitchenQual NA in pred. filling with most popular values
        features['KitchenQual'] = features['KitchenQual'].fillna(features['KitchenQual'].mode()[0])

        # FireplaceQu  NA in all. NA means No Fireplace
        features['FireplaceQu'] = features['FireplaceQu'].fillna('NoFP')

        # GarageType, GarageFinish, GarageQual  NA in all. NA means No Garage
        for col in ('GarageType', 'GarageFinish', 'GarageQual'):
            features[col] = features[col].fillna('NoGRG')

        # GarageCars  NA in pred. I suppose NA means 0
        features['GarageCars'] = features['GarageCars'].fillna(0.0)

        # SaleType NA in pred. filling with most popular values
        features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

        # Year and Month to categorical
        features['YrSold'] = features['YrSold'].astype(str)
        features['MoSold'] = features['MoSold'].astype(str)

        # Adding total sqfootage feature and removing Basement, 1st and 2nd floor features
        features['TotalSF'] = features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
        features.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
        return features
    if(option == "mean"):
        for i in my_frame.columns:
            my_frame[i] = my_frame[i].fillna(my_frame[i].dropna().mean())
        return my_frame

How to handle catagoricals

In [52]:
def catagorical(my_frame, option):
    if(option == "one_hot"):
        cats = [c for c, d in zip(my_frame.columns, my_frame.dtypes) if str(d) == 'object']
        onehotvals = utils.get_onehots(my_frame, cats)
        utils.set_onehots(my_frame, onehotvals, drop=True)
        return my_frame

In [53]:
def results(my_frame, option):
    if option == "sklearn_rfr":
        clf = RandomForestRegressor()
    if option == "elastic":
        clf = ElasticNet()
        
    print(utils.run_cross_val(clf, my_frame))

In [54]:
def classify():
    train = utils.get_train_data(int_fillna_cols=False, one_hot=False)

    train = do_nan(train, "custom")

    train = catagorical(train, "one_hot")

    results(train, "elastic")
    
classify()

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/anaconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.........................................................